In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))


In [3]:
import numpy as np
import pandas as pd
import optuna
import re
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.metrics import roc_auc_score,accuracy_score,recall_score,f1_score,precision_score
from optimization.optimize_ensemble import *

c:\Users\DELL\Desktop\project\Customer Churn Related Things\customer_churn_prediction\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
transformed_featured_train_set = pd.read_csv(r"../data/processed/transformed_featured_train_set.csv")
transformed_featured_val_set = pd.read_csv(r"../data/processed/transformed_featured_val_set.csv")
transformed_featured_test_set = pd.read_csv(r"../data/processed/transformed_featured_test_set.csv")
transformed_featured_final_train_set = pd.read_csv(r"../data/processed/transformed_featured_final_train_set.csv")
lgb_featured_study = pd.read_csv(r"../data/processed/lgb_featured_study.csv")
xgb_featured_study = pd.read_csv(r"../data/processed/xgb_featured_study.csv")
catboost_featured_study = pd.read_csv(r"../data/processed/catboost_featured_study.csv")
nn_featured_study = pd.read_csv(r"../data/processed/nn_featured_study.csv")



In [7]:

def objective(trial):
    X_train, y_train = transformed_featured_train_set.drop('Churn', axis=1), transformed_featured_train_set['Churn']  # Extract training features and labels
    X_val, y_val = transformed_featured_val_set.drop('Churn', axis=1), transformed_featured_val_set['Churn']  # Extract validation features and labels
    
    # Sample indices for hyperparameters from different models
    lgb_study = drop_unnessesary_columns(lgb_featured_study.copy())  # Prepare LightGBM study data
    xgb_study = drop_unnessesary_columns(xgb_featured_study.copy())  # Prepare XGBoost study data
    cat_study = drop_unnessesary_columns(catboost_featured_study.copy())  # Prepare CatBoost study data
    nn_study = drop_unnessesary_columns(nn_featured_study.copy())  # Prepare Neural Network study data
    
    lgb_idx = trial.suggest_int('lgb_idx', 0, len(lgb_study) - 1)  # Sample index for LightGBM hyperparameters
    xgb_idx = trial.suggest_int('xgb_idx', 0, len(xgb_study) - 1)  # Sample index for XGBoost hyperparameters
    cat_idx = trial.suggest_int('cat_idx', 0, len(cat_study) - 1)  # Sample index for CatBoost hyperparameters
    nn_idx = trial.suggest_int('nn_idx', 0, len(nn_study) - 1)  # Sample index for Neural Network hyperparameters
    
    # Fetch hyperparameters and clean them
    lgb_params = clean_hyperparameters(lgb_study.iloc[lgb_idx].to_dict())  # Get LightGBM parameters
    xgb_params = clean_hyperparameters(xgb_study.iloc[xgb_idx].to_dict())  # Get XGBoost parameters
    cat_params = clean_hyperparameters(cat_study.iloc[cat_idx].to_dict())  # Get CatBoost parameters
    nn_params = clean_hyperparameters(nn_study.iloc[nn_idx].to_dict())  # Get Neural Network parameters
    
    # Set additional fixed hyperparameters
    lgb_params['verbose'] = -1  # Silence LightGBM output
    xgb_params['verbose'] = 0  # Silence XGBoost output
    cat_params['early_stopping_rounds'] = 3000  # Set early stopping rounds for CatBoost
    cat_params['iterations'] = 200  # Set number of iterations for CatBoost

    # Weights for the voting classifier
    lgb_weight = trial.suggest_float('lgb_weight', 0.1, 1.0)  # Suggest weight for LightGBM
    xgb_weight = trial.suggest_float('xgb_weight', 0.1, 1.0)  # Suggest weight for XGBoost
    cat_weight = trial.suggest_float('cat_weight', 0.1, 1.0)  # Suggest weight for CatBoost
    nn_weight = trial.suggest_float('nn_weight', 0.1, 1.0)  # Suggest weight for Neural Network

    weights = {
        'lgb': lgb_weight,
        'xgb': xgb_weight,
        'cat': cat_weight,
        'nn': nn_weight
    }  # Store weights in a dictionary
    
    #Let’s give these models some workout time. First up, LightGBM. 
    #It's like the gym but for data—let’s see if it can lift those predictions high!
    
    # Train and predict with LightGBM
    lgb_train = lgb.Dataset(X_train, label=y_train)  # Prepare LightGBM dataset
    lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=100)  # Train LightGBM model
    lgb_preds = lgb_model.predict(X_val)  # Predict on validation set with LightGBM
    
    #Phew, LightGBM is done. Now let’s see if XGBoost can boost our mood... or just our predictions!"
    
    # Train and predict with XGBoost
    xgb_model = xgb.XGBClassifier(**xgb_params)  # Initialize XGBoost model with parameters
    xgb_model.fit(X_train, y_train)  # Train XGBoost model
    xgb_preds = xgb_model.predict_proba(X_val)[:, 1]  # Predict on validation set with XGBoost and get probabilities

    # Train and predict with CatBoost
    cat_model = cb.CatBoostClassifier(**cat_params, verbose=0)  # Initialize CatBoost model with parameters
    cat_model.fit(X_train, y_train)  # Train CatBoost model
    cat_preds = cat_model.predict_proba(X_val)[:, 1]  # Predict on validation set with CatBoost and get probabilities

    # Train and predict with Neural Network using TensorFlow/Keras
    nn_model = create_nn_model(nn_params, transformed_featured_train_set.shape[1] - 1)  # Create Neural Network model
    nn_model.fit(X_train, y_train, epochs=50, batch_size=int(nn_params['batch_size']), verbose=0)  # Train Neural Network model
    nn_preds = nn_model.predict(transformed_featured_val_set.drop('Churn', axis=1)).ravel()  # Predict on validation set with Neural Network

    # Combine predictions using weighted soft voting
    predictions = {
        'lgb': lgb_preds,
        'xgb': xgb_preds,
        'cat': cat_preds,
        'nn': nn_preds
    }  # Store predictions in a dictionary
    combined_preds = weighted_voting(predictions, weights)  # Perform weighted voting to combine predictions
    preds_digits = [1 if pred >= 0.4 else 0 for pred in combined_preds]  # Convert probabilities to binary predictions with a threshold of 0.4
    
    # Calculate evaluation metrics
    roc_auc = roc_auc_score(y_val, combined_preds)  # Calculate ROC AUC score
    f1 = f1_score(y_val, preds_digits)  # Calculate F1 score
    recall = recall_score(y_val, preds_digits)  # Calculate recall score
    accuracy = accuracy_score(y_val, preds_digits)  # Calculate accuracy score
    weighted_recall = 0.65 * recall + 0.35 * f1  # Calculate weighted recall combining recall and F1 score
    prec = precision_score(y_val, preds_digits)  # Calculate precision score
    
    # Store metrics as trial user attributes
    trial.set_user_attr('roc', roc_auc)  # Store ROC AUC score in the study
    trial.set_user_attr('f1', f1)  # Store F1 score in the study object
    trial.set_user_attr('accuracy', accuracy)  # Store accuracy score
    trial.set_user_attr('recall', recall)  # Store recall score
    trial.set_user_attr('precision', prec)  # Store precision score
    
    return weighted_recall  # Return weighted recall as the objective value for optimization

In [8]:
run_ensemble_trials = True

In [9]:

if run_ensemble_trials:
    # Create Optuna study and optimize
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=5 , n_jobs =-1)

    # Best trial
    best_trial = study.best_trial
    print(f'Best metric: {best_trial.value}')
    print('Best hyperparameters and weights:', best_trial.params)
    trials = study.trials

    # Extract trial data
    data = {
        'trial_number': [trial.number for trial in trials],
        'value': [trial.value for trial in trials],
        'params': [trial.params for trial in trials],
        'datetime_start': [trial.datetime_start for trial in trials],
        'datetime_complete': [trial.datetime_complete for trial in trials],
        'f1': [trial.user_attrs.get('f1', None) for trial in trials],
        'accuracy': [trial.user_attrs.get('accuracy', None) for trial in trials],
        'roc': [trial.user_attrs.get('roc', None) for trial in trials],
        'recall': [trial.user_attrs.get('recall', None) for trial in trials],
        'precision': [trial.user_attrs.get('precision', None) for trial in trials]
        
    }

    # Convert to DataFrame
    ensemble_results_df = pd.DataFrame(data)
    
else:
    ensemble_results_df = pd.read_csv(r"../data/processed/ensemble_study.csv")
    ensemble_results_df = ensemble_results_df.drop(columns="Unnamed: 0")

[I 2024-08-29 09:03:20,642] A new study created in memory with name: no-name-166969c0-5793-4e49-9d27-10c4fa159205
c:\Users\DELL\Desktop\project\Customer Churn Related Things\customer_churn_prediction\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\DELL\Desktop\project\Customer Churn Related Things\customer_churn_prediction\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\DELL\Desktop\project\Customer Churn Related Things\customer_churn_predictio

27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


[I 2024-08-29 09:04:39,669] Trial 3 finished with value: 0.5786922703478554 and parameters: {'lgb_idx': 83, 'xgb_idx': 71, 'cat_idx': 17, 'nn_idx': 47, 'lgb_weight': 0.9203079150266962, 'xgb_weight': 0.5512132048293996, 'cat_weight': 0.3528436089976743, 'nn_weight': 0.4561744578200917}. Best is trial 3 with value: 0.5786922703478554.
[I 2024-08-29 09:04:39,710] Trial 4 finished with value: 0.54270586492891 and parameters: {'lgb_idx': 85, 'xgb_idx': 6, 'cat_idx': 70, 'nn_idx': 22, 'lgb_weight': 0.29580315845167665, 'xgb_weight': 0.7512250283981811, 'cat_weight': 0.588684365372811, 'nn_weight': 0.8407556198551432}. Best is trial 3 with value: 0.5786922703478554.
[I 2024-08-29 09:04:39,880] Trial 0 finished with value: 0.6007034632034631 and parameters: {'lgb_idx': 90, 'xgb_idx': 25, 'cat_idx': 60, 'nn_idx': 40, 'lgb_weight': 0.9641079816025739, 'xgb_weight': 0.43383190908494507, 'cat_weight': 0.4191301008310486, 'nn_weight': 0.819552639978684}. Best is trial 0 with value: 0.6007034632034

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


[I 2024-08-29 09:04:47,836] Trial 2 finished with value: 0.5631001170960188 and parameters: {'lgb_idx': 73, 'xgb_idx': 59, 'cat_idx': 88, 'nn_idx': 86, 'lgb_weight': 0.22688645252407963, 'xgb_weight': 0.154703130836274, 'cat_weight': 0.468488194104861, 'nn_weight': 0.9977956230814264}. Best is trial 0 with value: 0.6007034632034631.


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


[I 2024-08-29 09:04:56,312] Trial 1 finished with value: 0.6047973825291706 and parameters: {'lgb_idx': 78, 'xgb_idx': 36, 'cat_idx': 93, 'nn_idx': 40, 'lgb_weight': 0.7214605459161612, 'xgb_weight': 0.7072014936115618, 'cat_weight': 0.6044746694563032, 'nn_weight': 0.39285572034959515}. Best is trial 1 with value: 0.6047973825291706.


Best metric: 0.6047973825291706
Best hyperparameters and weights: {'lgb_idx': 78, 'xgb_idx': 36, 'cat_idx': 93, 'nn_idx': 40, 'lgb_weight': 0.7214605459161612, 'xgb_weight': 0.7072014936115618, 'cat_weight': 0.6044746694563032, 'nn_weight': 0.39285572034959515}


In [10]:
# model's performance which has highest weighted recall
ensemble_results_df.iloc[ensemble_results_df['value'].idxmax()]

trial_number                                                         1
value                                                         0.604797
params               {'lgb_idx': 78, 'xgb_idx': 36, 'cat_idx': 93, ...
datetime_start                              2024-08-29 09:03:20.679105
datetime_complete                           2024-08-29 09:04:56.311433
f1                                                            0.600442
accuracy                                                      0.785545
roc                                                           0.833842
recall                                                        0.607143
precision                                                     0.593886
Name: 1, dtype: object

In [11]:
# model's performance which has highest recall
ensemble_results_df.iloc[ensemble_results_df['recall'].idxmax()]

trial_number                                                         0
value                                                         0.600703
params               {'lgb_idx': 90, 'xgb_idx': 25, 'cat_idx': 60, ...
datetime_start                              2024-08-29 09:03:20.672777
datetime_complete                           2024-08-29 09:04:39.875382
f1                                                            0.588745
accuracy                                                      0.774882
roc                                                           0.833129
recall                                                        0.607143
precision                                                     0.571429
Name: 0, dtype: object

In [12]:
# model's performance which has highest precision
ensemble_results_df.iloc[ensemble_results_df['precision'].idxmax()]

trial_number                                                         3
value                                                         0.578692
params               {'lgb_idx': 83, 'xgb_idx': 71, 'cat_idx': 17, ...
datetime_start                              2024-08-29 09:03:20.690206
datetime_complete                           2024-08-29 09:04:39.669298
f1                                                            0.600473
accuracy                                                      0.799763
roc                                                           0.837414
recall                                                        0.566964
precision                                                     0.638191
Name: 3, dtype: object

In [14]:
ann_model = create_nn_model(clean_hyperparameters(nn_featured_study.iloc[87].to_dict()),input_shape=transformed_featured_train_set.drop(columns='Churn').shape[1])
ann_model.fit(transformed_featured_final_train_set.drop(columns='Churn'),transformed_featured_final_train_set['Churn'],epochs=20,validation_data=(transformed_featured_test_set.drop(columns='Churn'),transformed_featured_test_set['Churn']))

c:\Users\DELL\Desktop\project\Customer Churn Related Things\customer_churn_prediction\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - AUC: 0.5185 - loss: 55.7802 - val_AUC: 0.6178 - val_loss: 1.4507
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - AUC: 0.5035 - loss: 25.2308 - val_AUC: 0.5850 - val_loss: 1.4552
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - AUC: 0.5247 - loss: 15.1781 - val_AUC: 0.6264 - val_loss: 0.7400
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - AUC: 0.5039 - loss: 9.4261 - val_AUC: 0.6355 - val_loss: 0.6090
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - AUC: 0.5157 - loss: 5.7865 - val_AUC: 0.6906 - val_loss: 0.5487
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - AUC: 0.5320 - loss: 3.8098 - val_AUC: 0.7501 - val_loss: 0.6265
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - AUC: 0.5218 - loss: 2.3857 - val_AUC: 0.6555 - val_loss: 0.6458
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - AUC: 0.5276 - loss: 1.5796 - val_AUC: 0.7100 - val_loss: 0.6602
Epoch 9/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step 

In [21]:
#let's see final model's performance
preds = ann_model.predict(transformed_featured_test_set.drop(columns='Churn'))
print('roc_auc score is:-',roc_auc_score(transformed_featured_test_set['Churn'],preds))
preds_digits = [1 if pred >= 0.4 else 0 for pred in preds]
print('recall score is :-',recall_score(transformed_featured_test_set['Churn'],preds_digits))
print('precision score is:-',precision_score(transformed_featured_test_set['Churn'],preds_digits))

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
roc_auc score is:- 0.79433507099927
recall score is :- 0.0
precision score is:- 0.0
